In [1]:
%%capture
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Colab Notebooks/WhenPigsFlyContext/baselines/Detectron2

!python -m pip install pyyaml==5.1
# Detectron2 has not released pre-built binaries for the latest pytorch (https://github.com/facebookresearch/detectron2/issues/4053)
# so we install from source instead. This takes a few minutes.
!python -m pip install 'git+https://github.com/facebookresearch/detectron2.git'
!pip install ml-collections

In [7]:
%%capture
! pip3 install pickle5

In [10]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.utils.events import EventStorage
from detectron2.structures import Boxes, ImageList, Instances, pairwise_iou
import detectron2.data.transforms as T

# import pytorch
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets, models, transforms, utils
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms.functional import to_pil_image, to_tensor

from PIL import Image
from tqdm import tqdm, trange
from random import sample
import pickle
import pickle5 as pickle5
import copy
%matplotlib inline
from matplotlib import pyplot as plt
import sys
sys.path.append("..")
from SCEGRAM.SCEGRAM import SCEGRAM
from utils import *

In [3]:
# transform attention map from rgb to grayscale
transform_single = transforms.Compose(
    [
        transforms.Grayscale(num_output_channels=1),
        transforms.ToTensor()
    ]
) 

# change PIL image to pytorch tensor
img_transforms = transforms.Compose(
    [
        transforms.Resize((800, 1280)),
        transforms.ToTensor(),
    ]
)

# process predicted objectiveness logits to range(0, 1)
sigmoid = torch.nn.Sigmoid()

In [4]:
context_dir = "../../SCEGRAM/01scenes/01object_present"
target_dir = "../../SCEGRAM/02objects"
info_dir = "../../SCEGRAM/SCEGRAM_Database_scenes_objects.xlsx"
context_size, target_size = (224, 224), (224, 224)
dataset = SCEGRAM(info_dir, context_dir, target_dir, context_size, target_size, is_transform=False)

/usr/local/lib/python3.7/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [9]:
with open("../IVSN/[SCEGRAM]bin_idxs.pkl", "rb") as tf:
    bin_info = pickle5.load(tf) 

In [5]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
cfg.merge_from_file(model_zoo.get_config_file("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"))
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
# Find a model from detectron2's model zoo. You can use the https://dl.fbaipublicfiles... url as well
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml")
aug = T.ResizeShortestEdge(
            [cfg.INPUT.MIN_SIZE_TEST, cfg.INPUT.MIN_SIZE_TEST], cfg.INPUT.MAX_SIZE_TEST
        )

predictor = DefaultPredictor(cfg)
model = build_model(cfg)
DetectionCheckpointer(model).load(cfg.MODEL.WEIGHTS)

model_final_f10217.pkl: 178MB [00:05, 29.8MB/s]                           


{'__author__': 'Detectron2 Model Zoo'}

In [18]:
size, img_size, original_size = 48, (320, 512), (1024, 1280)
detectron2_CON_0_25, detectron2_CON_25_50 = [], []
detectron2_INCON_0_25, detectron2_INCON_25_50 = [], []
scanpath, detectron2_attention_map = {}, {}
selected_imgs = bin_info['con_(0, 25]'].tolist() + bin_info['con_(25, 50]'].tolist() + bin_info['incon_(0, 25]'].tolist() + bin_info['incon_(25, 50]'].tolist()

for id in trange(0, len(dataset)):
    if id not in selected_imgs:
        continue

    img, _, bbox_relative, category = dataset[id]
    # get the target bounding box
    tg_loc = bbox_cordinates(bbox_relative, img_size[1], img_size[0])

    # get the img path
    image = aug.get_transform(img).apply_image(img)
    img_tensor = torch.as_tensor(image.astype("float32").transpose(2, 0, 1)).cuda()
    img_tensor = transforms.Resize(original_size)(img_tensor)
    imgList = ImageList.from_tensors([img_tensor])

    # compute the attention map
    model.eval()
    with torch.no_grad():
        features = model.backbone(img_tensor.unsqueeze(0))
        mask_features = [features[f] for f in features.keys()]
        pred_objectness_logits, _ = model.proposal_generator.rpn_head(mask_features)

    attention_map = torch.zeros((3, 320, 512)).detach().cuda()
    for i in range(len(pred_objectness_logits)):
        logits = transforms.Resize(img_size)(pred_objectness_logits[i].squeeze(0)).cuda()
        logits = sigmoid(logits)
        attention_map += logits

    attention_map = attention_map/len(pred_objectness_logits)
    attention_map = torch.divide(attention_map, attention_map.max())
    attention_map = transform_single(to_pil_image(attention_map))

    # save the attention map
    detectron2_attention_map[id] = copy.deepcopy(attention_map)

    search_num, path = searchProcesswithPath(tg_loc, attention_map, img_size, size)
    scanpath[id] = path
    
    if id in bin_info['con_(0, 25]'].tolist():
        detectron2_CON_0_25.append(search_num)
    elif id in bin_info['con_(25, 50]'].tolist():
        detectron2_CON_25_50.append(search_num)

    elif id in bin_info['incon_(0, 25]'].tolist():
        detectron2_INCON_0_25.append(search_num)
    elif id in bin_info['incon_(25, 50]'].tolist():
        detectron2_INCON_25_50.append(search_num)

    print('detectron2_' + str(id) + ': ' + str(search_num), end = '\t')

detectron2_CON_res = detectron2_CON_0_25 + detectron2_CON_25_50
detectron2_INCON_res = detectron2_INCON_0_25 + detectron2_INCON_25_50
detectron2_res = detectron2_CON_res + detectron2_INCON_res

  0%|          | 1/372 [00:00<01:26,  4.30it/s]

detectron2_0: 5	

  1%|          | 3/372 [00:00<01:17,  4.78it/s]

detectron2_1: 2	detectron2_2: 14	

  1%|▏         | 5/372 [00:01<01:13,  5.00it/s]

...

detectron2_364: 71	

 99%|█████████▉| 370/372 [01:06<00:00,  3.97it/s]

detectron2_369: 2	

100%|██████████| 372/372 [01:06<00:00,  5.56it/s]

detectron2_371: 43	

In [19]:
np.mean(detectron2_res), np.mean(detectron2_CON_res), np.mean(detectron2_INCON_res)

(15.684491978609625, 5.59375, 17.767741935483873)

In [20]:
len(detectron2_res) == len(detectron2_CON_res) + len(detectron2_INCON_res)

True

In [23]:
def sampleIncon(incon_bin_result, con_bin_result, times):
    sample_times = times
    nums = len(con_bin_result)
    res = np.array([0.0] * 25)
    
    for id in range(sample_times):
        temp = sample(incon_bin_result, nums)
        temp_accu = model_performance(temp, len(temp))
        res += np.array(temp_accu[:25])

    return (res/sample_times).tolist()

def balanced_accu(res_con, res_incon):
    res = []
    for i in range(25):
        res.append((res_con[i]+res_incon[i])/2)

    return res

In [24]:
times = 100
detectron2_CON_accu = model_performance(detectron2_CON_res, len(detectron2_CON_res))
detectron2_INCON_accu = sampleIncon(detectron2_INCON_res, detectron2_CON_res, times)
detectron2_accu = balanced_accu(detectron2_CON_accu, detectron2_INCON_accu)
detectron2_accu[:11], detectron2_CON_accu[:11], detectron2_INCON_accu[:11]

([0.0,
  0.17375000000000002,
  0.3678125,
  0.45734375,
  0.50734375,
  0.63359375,
  0.65703125,
  0.66859375,
  0.70953125,
  0.740625,
  0.7557812500000001],
 [0,
  0.25,
  0.46875,
  0.59375,
  0.625,
  0.8125,
  0.8125,
  0.8125,
  0.84375,
  0.84375,
  0.84375],
 [0.0,
  0.0975,
  0.266875,
  0.3209375,
  0.3896875,
  0.4546875,
  0.5015625,
  0.5246875,
  0.5753125,
  0.6375,
  0.6678125])

In [26]:
detectron2_SCEGRAM_res = {}
detectron2_SCEGRAM_res['combined_accu'] = detectron2_accu
detectron2_SCEGRAM_res['con_accu'] = detectron2_CON_accu
detectron2_SCEGRAM_res['incon_accu'] = detectron2_INCON_accu
detectron2_SCEGRAM_res['con_[0,25)'] = detectron2_CON_0_25
detectron2_SCEGRAM_res['con_[25,50)'] = detectron2_CON_25_50
detectron2_SCEGRAM_res['incon_[0,25)'] = detectron2_INCON_0_25
detectron2_SCEGRAM_res['incon_[25,50)'] = detectron2_INCON_25_50
detectron2_SCEGRAM_res['scanpath'] = scanpath
detectron2_SCEGRAM_res['attention_map'] = detectron2_attention_map

In [27]:
with open("../results/SCEGRAM/SCEGRAM(invariant_bin1_2)_detectron2_res.pkl", "wb") as tf:
    pickle.dump(detectron2_SCEGRAM_res, tf)